# Programação Orientada a Objetos (POO)
## Tema 8 – Aplicações e Serviços Web

Jaime A. Martins

(CEOT/ISE/UAlg - jamartins@ualg.pt)


###### Autor: Jaime Martins

## Criar web apps com Flask

### **v1** – Uma web app simples

Vamos criar uma web app simples com Flask, que apenas retorna uma página HTML estática (uma string "Hello, World!"):

``` python
# web-app1.py
from flask import Flask

app = Flask(__name__)


@app.route("/")
@app.route("/index")
def index():
    return "Hello, World!"


if __name__ == "__main__":
    app.run(debug=True)
```

Podemos correr o servidor Flask num terminal do Commmand Prompt:

In [ ]:
import os

os.environ['FLASK_APP'] = 'web-app-v1.py'
custom_port = 8080

os.system(f'start cmd.exe @cmd /c "flask run --port {custom_port}"')

del os.environ["FLASK_APP"]

### **v2** – Uma web app dinâmica

Na v2 vamos modificar a web app para usar código HTML em vez de uma string estática, e preencher o HTML com dados dinâmicos:

``` python
# web-app2.py
from flask import Flask

app = Flask(__name__)

things = [
    {
        "id": 1,
        "name": "Prometheus",
    }
]


@app.route("/")
@app.route("/index")
def index():
    html = f"""
        <html>
            <head>
                <title>Things (v2)</title>
            </head>
            <body>
                <h1>Thing: {things[0]['name']}!</h1>
            </body>
        </html>
        """

    return html


if __name__ == "__main__":
    app.run(debug=True)
```

Podemos correr o servidor Flask num terminal do Commmand Prompt:

In [ ]:
import os

os.environ['FLASK_APP'] = 'web-app-v2.py'
custom_port = 8080

os.system(f'start cmd.exe @cmd /c "flask run --port {custom_port}"')

del os.environ["FLASK_APP"]

### **v3** – Uma web app dinâmica com template Jinja2 (simples)

Em vez de estarmos a usar strings com código HTML:
* Podemos recriar a web app anterior usando um template `Jinja2`, que é um ficheiro HTML onde se criam variáveis que irão ser preenchidas com dados dinâmicos.
* Em sintaxe Jinja, as variáveis são colocadas entre chavetas duplas `{{ }}`, para indicar que devem ser posteriormente substituídas pelo seu valor.

``` html
{# templates/thing.html #}
<!DOCTYPE html>
<html>
    <head>
        <title>{{ title }}</title>
    </head>
    <body>
        <h1>Thing: {{ thing.name }}!</h1>
    </body>
</html>

```

Neste caso, o template é guardado no ficheiro `thing.html` e é preenchido (*rendered*) usando o método `render_template()` do Flask.

O `render_template()` recebe o nome do template e os valores das variáveis que devem ser substituídas.
O Flask irá procurar o template `thing.html` na pasta `templates`.

``` python
# web-app3.py
from flask import Flask, render_template

app = Flask(__name__)

things = [
    {
        "id": 1,
        "name": "Prometheus",
    }
]


@app.route("/")
@app.route("/index")
def index():
    return render_template("thing.html", title="Things (v3)", thing=things[0])


if __name__ == "__main__":
    app.run(debug=True)
```

Podemos correr o servidor Flask num terminal do Commmand Prompt:

In [ ]:
import os

os.environ['FLASK_APP'] = 'web-app-v3.py'
custom_port = 8080

os.system(f'start cmd.exe @cmd /c "flask run --port {custom_port}"')

del os.environ["FLASK_APP"]

### **v4** – Uma web app dinâmica com template Jinja2 (avançado)

E se quisermos mostrar uma lista de coisas (things) em vez de apenas uma?

Podemos usar um ciclo `for` diretamente dentro do template Jinja2 para iterar sobre uma lista de things:

``` html
{# templates/things-lista.html #}
<!DOCTYPE html>

{# isto é um comentario #}
{# https://jinja.palletsprojects.com/en/3.0.x/templates/ #}
{#
    {{ }} - The tag marking the beginning/ending of a print statement.
    {% %} - The tag marking the beginning/ending of a control block
#}

<html lang="en">
    <head>
        <title>{{ title }}</title>
    </head>
    <body>
        <h1>Things:</h1>
        {% for thing in things %}
            <div>
                <h2>{{ thing.name }}</h2>
                <p>Localizado em {{ thing.local }}. Tem os sensores:</p>
                {% for sensor in thing.sensors %}
                    <div>
                        {{ sensor.sensor_name }}
                    </div>
                {% endfor %}
            </div>
        {% endfor %}
    </body>
</html>
```

1. O ciclo `for` é definido com a diretiva `for` e termina na diretiva `endfor`.
1. O ciclo `for` itera sobre a lista de `things` e, para cada iteração, a variável `thing` é preenchida com o valor da iteração atual.
1. Dentro do ciclo `for`, podemos usar a variável `thing` para aceder aos atributos da mesma.
1. O ciclo `for` nested itera sobre a lista de sensores de cada `thing`.

``` python
# web-app-v4.py
from flask import Flask, render_template

app = Flask(__name__)

things = [
    {
        "id": 1,
        "name": "Prometheus",
        "local": "@lab. 163 / ISE /UAlg",
        "sensors": [
            {"sensor_name": "mem_sensor", "units": "percent"},
            {"sensor_name": "cpu_sensor", "units": "percent"},
        ],
    },
    {
        "id": 2,
        "name": "Zeus",
        "local": "@lab. 163 / ISE /UAlg",
        "sensors": [
            {"sensor_name": "temperature", "units": "numerical"},
            {"sensor_name": "humidity", "units": "percent"},
        ],
    },
]


@app.route("/")
@app.route("/index")
def index():
    return render_template("things-lista.html", title="Things (v4)", things=things)


if __name__ == "__main__":
    app.run(debug=True)
```

Podemos correr o servidor Flask num terminal do Commmand Prompt:

In [ ]:
import os

os.environ['FLASK_APP'] = 'web-app-v4.py'
custom_port = 8080

os.system(f'start cmd.exe @cmd /c "flask run --port {custom_port}"')

del os.environ["FLASK_APP"]

### **v5** – Uma web app dinâmica com template Jinja2 (estendido)

O template HTML pode ser dividido em vários ficheiros, usando a diretiva `extends`, que nos permite reutilizar templates e evitar duplicação de código.

Primeiro, vamos criar um template `base.html` que contém o código HTML comum a todas as páginas:

``` html
{# templates/base.html #}
<!DOCTYPE html>
<html>
    <head>
        <title>
            {% block _title_ %} Title {% endblock %}
        </title>
    </head>
    <body>
        <hr>
        <div>
            <a href="/index">Home</a> ...
        </div>
        <hr>
        {% block _content_ %}
        {% endblock %}
    </body>
</html>

```

Os blocos `{% block <nome> %} {% endblock %}` assinalam os blocos que podem ser substituídos nos templates filhos.

E em seguida vamos criar um modelo `things-lista-v2.html` que herda do ficheiro `base.html`, através da diretiva `extends`:

``` html
{# templates/things-lista-v2.html #}
<!DOCTYPE html>
{% extends "base.html" %}
{# https://jinja.palletsprojects.com/en/3.0.x/templates/#template-inheritance #}

{% block _title_ %}
    {{ title }}
{% endblock _title_ %}

{% block _content_ %}
    {% for thing in things %}
        <div>
            <p>{{ thing.name }} located at {{ thing.local }} has sensors:</p>
            {% for sensor in thing.sensors %}<div>{{ sensor.sensor_name }} ({{ sensor.units }})</div>{% endfor %}
        </div>
    {% endfor %}
{% endblock _content_ %}
```

E finalmente a app Flask, idêntica à anterior excepto no nome do template invoocado:

``` python
# web-app-v5.py
from flask import Flask, render_template

app = Flask(__name__)

things = [
    {
        "id": 1,
        "name": "Prometheus",
        "local": "@lab. 163 / ISE /UAlg",
        "sensors": [
            {"sensor_name": "mem_sensor", "units": "percent"},
            {"sensor_name": "cpu_sensor", "units": "percent"},
        ],
    },
    {
        "id": 2,
        "name": "Zeus",
        "local": "@lab. 163 / ISE /UAlg",
        "sensors": [
            {"sensor_name": "temperature", "units": "numerical"},
            {"sensor_name": "humidity", "units": "percent"},
        ],
    },
]


@app.route("/")
@app.route("/index")
def index():
    return render_template("things-lista-v2.html", title="Things (v5)", things=things)


if __name__ == "__main__":
    app.run(debug=True)
```

Podemos correr o servidor Flask num terminal do Commmand Prompt:

In [ ]:
import os

os.environ['FLASK_APP'] = 'web-app-v5.py'
custom_port = 8080

os.system(f'start cmd.exe @cmd /c "flask run --port {custom_port}"')

del os.environ["FLASK_APP"]

In [ ]:
# Clean up the environment variable, if it still exists
if os.environ.get("FLASK_APP"):
    del os.environ["FLASK_APP"]